In [1]:
import pandas as pd
import geopandas as gpd
import rasterio
import rasterio.sample
import rioxarray
import plotly.io as pio
pio.renderers.default = 'vscode' # Or 'notebook', 'browser', 'jupyterlab' etc.

In [2]:
import plotly.express as px
import plotly.io as pio
# pio.renderers.default = 'notebook' # Or 'vscode', 'browser', etc. - uncomment and set if needed

print("Attempting minimal Plotly map...")
try:
    fig = px.scatter_mapbox(lat=[0], lon=[0], zoom=1) # Plot a single point at 0,0
    fig.update_layout(mapbox_style="open-street-map")
    fig.show()
    print("Minimal map should have displayed.")
except Exception as e:
    print(f"Error during minimal map test: {e}")

Attempting minimal Plotly map...


C:\Users\yuant\AppData\Local\Temp\ipykernel_19652\2312376925.py:7: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



Minimal map should have displayed.


In [5]:
tif_file_path = 'humansettle_data/GHS_BUILT_S_E2020_GLOBE_R2023A_54009_1000_V1_0_R7_C17.tif'

In [6]:
import rasterio
import rasterio.transform
import plotly.express as px
from pyproj import Transformer
import numpy as np
import pandas as pd
import plotly.io as pio
import traceback 

# --- Parameters ---
value_threshold = 0 # Only plot pixels with value strictly greater than this
max_points_to_plot = 1000 # Limit the number of points for performance

try:
    with rasterio.open(tif_file_path) as src:
        print(f"Reading raster: {tif_file_path}")
        # CORRECTED PRINT STATEMENT: Use src.dtypes[0]
        print(f"CRS: {src.crs}, Shape: {src.shape}, Dtype: {src.dtypes[0]}")

        # 1. Read Raster Data (read band 1)
        data = src.read(1)
        print(f"NoData value: {src.nodata}")
        print(f"Data min: {np.min(data)}, max: {np.max(data)}, mean: {np.mean(data)}")
        transform = src.transform
        native_crs = src.crs

        # 2. Identify Relevant Pixels
        # Find indices (row, col) where data > threshold
        rows, cols = np.where(data > value_threshold)
        values = data[rows, cols]
        num_found = len(rows)
        print(f"Found {num_found} pixels with value > {value_threshold}")

        if num_found == 0:
            print("No relevant pixels found based on threshold. Exiting.")
            exit()

        # Optional: Sample if too many points found
        if num_found > max_points_to_plot:
            print(f"Sampling down to {max_points_to_plot} points...")
            indices = np.arange(num_found)
            sampled_indices = np.random.choice(indices, max_points_to_plot, replace=False)
            rows = rows[sampled_indices]
            cols = cols[sampled_indices]
            values = values[sampled_indices]

        # 3. Calculate Center Coordinates (Native CRS)
        # Use transform.xy() which applies the affine transform correctly
        # It takes row, col indices - provide center offset (0.5)
        xs, ys = transform * (cols + 0.5, rows + 0.5) # Get native X, Y coords

        # 4. Transform to Lat/Lon (WGS84)
        wgs84_crs = "EPSG:4326"
        transformer = Transformer.from_crs(str(native_crs), wgs84_crs, always_xy=True)
        lons, lats = transformer.transform(xs, ys)

        # 5. Create DataFrame
        df_plot = pd.DataFrame({
            'latitude': lats,
            'longitude': lons,
            'settlement_value': values
        })
        print(f"Created DataFrame with {len(df_plot)} points for plotting.")
        # print(df_plot.head()) # Optional: Inspect the data

        # 6. Plot with Plotly Express
        print("Creating Plotly map...")
        fig = px.scatter_mapbox(df_plot,
                                lat="latitude",
                                lon="longitude",
                                color="settlement_value", # Color points by settlement value
                                size_max=5,          # Adjust marker size if needed (can remove if not needed)
                                opacity=0.7,
                                # range_color=[values.min(), values.max()], # Set color range if desired
                                color_continuous_scale=px.colors.sequential.Viridis, # Choose a colorscale
                                zoom=10, # Adjusted zoom slightly, maybe start closer for 10m data
                                center={"lat": np.mean(lats), "lon": np.mean(lons)}, # Use numpy mean for center calculation
                                title=f"Settlement Pixel Centers (> {value_threshold})\n{tif_file_path}"
                               )

        fig.update_layout(mapbox_style="carto-positron") # Use a light basemap
        fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
        fig.update_layout(
          height=700, # Set the figure height in pixels
          # You can change the value (e.g., 700) to make it taller or shorter
          margin={"r":0,"t":30,"l":0,"b":0}
          )

        print("Showing plot...")
        fig.show()
        print("Plot display attempted.")


except rasterio.RasterioIOError as e:
    print(f"Error opening or reading file: {e}")
except FileNotFoundError:
    print(f"Error: File not found at {tif_file_path}")
except Exception as e:
    # Print the specific traceback for better debugging
    print(f"An unexpected error occurred during processing or plotting:")
    traceback.print_exc() # This will print the full error details

Reading raster: humansettle_data/GHS_BUILT_S_E2020_GLOBE_R2023A_54009_1000_V1_0_R7_C17.tif
CRS: ESRI:54009, Shape: (1000, 700), Dtype: uint32
NoData value: 4294967295.0
Data min: 0, max: 4294967295, mean: 613567015.6984457
Found 134907 pixels with value > 0
Sampling down to 1000 points...
Created DataFrame with 1000 points for plotting.
Creating Plotly map...


C:\Users\yuant\AppData\Local\Temp\ipykernel_66228\356529412.py:68: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



Showing plot...


Plot display attempted.


In [2]:
# --- 1. Load your CSV data ---
csv_path = 'data/test.csv' # Replace with your file path
df = pd.read_csv(csv_path)

# --- 2. Create a GeoDataFrame ---
# Assuming your lat/lon are in WGS84 (EPSG:4326)
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df.lon, df.lat),
    crs="EPSG:4326" # WGS84 coordinate system
)

# --- 3. Download and Specify Raster Path ---
# Download the relevant GHSL GeoTIFF (e.g., for Angola, for the year closest to 2006)
# Example: Let's say you downloaded the 100m Built-up Surface for 2005
ghsl_raster_path = 'humansettle_data/GHS_BUILT_S_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C17.tif' # Replace

# --- 4. Open Raster and Extract Values ---

# Option A: Using rasterio
coords = [(p.x, p.y) for p in gdf.geometry] # List of (lon, lat) tuples

with rasterio.open(ghsl_raster_path) as src:
    # Ensure points are in the same CRS as the raster
    # GHSL is often in Mollweide (EPSG:54009), check the raster's CRS: src.crs
    if gdf.crs != src.crs:
        print(f"Reprojecting points from {gdf.crs} to {src.crs}")
        gdf_proj = gdf.to_crs(src.crs)
        coords_proj = [(p.x, p.y) for p in gdf_proj.geometry]
    else:
        coords_proj = coords

    # Sample the raster at the projected coordinates
    # 'sample_gen' returns a generator, convert it to a list of numpy arrays
    # Each array contains the raster value(s) for the corresponding point
    extracted_values_gen = src.sample(coords_proj)
    # Assuming single band raster, extract the first value for each point
    extracted_values = [val[0] for val in extracted_values_gen]

# Option B: Using rioxarray (often simpler)
rds = rioxarray.open_rasterio(ghsl_raster_path, masked=True).squeeze()

# Ensure points are in the same CRS as the raster
if gdf.crs != rds.rio.crs:
    print(f"Reprojecting points from {gdf.crs} to {rds.rio.crs}")
    gdf_proj = gdf.to_crs(rds.rio.crs)
else:
    gdf_proj = gdf

# Prepare coordinates for xarray selection
x_coords = xr.DataArray(gdf_proj.geometry.x, dims="points")
y_coords = xr.DataArray(gdf_proj.geometry.y, dims="points")

# Select values at point locations (nearest neighbor is default)
extracted_values_da = rds.sel(x=x_coords, y=y_coords, method="nearest")
extracted_values = extracted_values_da.values


# --- 5. Add Extracted Values to DataFrame ---
df['ghsl_builtup'] = extracted_values # Or use a more descriptive name

# --- 6. Use for Imputation ---
# Now 'ghsl_builtup' column can be used as a feature in your imputation model
# (e.g., using scikit-learn's IterativeImputer, KNNImputer, or custom methods)
print(df.head())

# Example: Check correlation with 'iwi' (if that's what you're imputing)
# print(df[['iwi', 'ghsl_builtup']].corr())

Reprojecting points from EPSG:4326 to ESRI:54009
Reprojecting points from EPSG:4326 to ESRI:54009


NameError: name 'xr' is not defined